We have to predict closing price of stocks of indian stock market.


In [2]:
import pandas as pd
pd.plotting.register_matplotlib_converters()


* We have taken data from google finance
* This project involves forecasting stocks closing price


In [5]:
df=pd.read_csv("./data/nifty50.csv",index_col="Date",parse_dates=["Date"])
df.head()

,Open,High,Low,Close
Date,,,,
07/05/2021 15:30:00,14816.85,14863.05,14765.5,14823.15
10/05/2021 15:30:00,14928.25,14966.90,14892.5,14942.35
11/05/2021 15:30:00,14789.70,14900.00,14771.4,14850.75
12/05/2021 15:30:00,14823.55,14824.05,14649.7,14696.50
14/05/2021 15:30:00,14749.40,14749.65,14591.9,14677.80
